# 🔥 VTrackAI Studio - One Click GPU

**T4 GPU + HF_TOKEN → Run All → Open UI link**

1. Runtime → **T4 GPU**
2. Sidebar → **Secrets** → `HF_TOKEN`
3. **Ctrl+F9** → Wait → Open Vite link

In [ ]:
# 1. GPU + Install (2min)
import torch
print(f'🔥 CUDA: {torch.cuda.is_available()}')

!rm -rf /content/vtrack-ai-studio /content/sam3
!pip install -q torch torchvision --index-url https://download.pytorch.org/whl/cu121
!pip install -q fastapi uvicorn[standard] python-multipart huggingface_hub nest-asyncio
!apt -qq update && apt -yqq install nodejs npm > /dev/null

# Clone + SAM3
!git clone https://github.com/facebookresearch/sam3
!pip install -e ./sam3 --quiet
!git clone https://github.com/Amitro123/vtrack-ai-studio
%cd /content/vtrack-ai-studio/backend
!pip install -r requirements.txt -q

# HF + Checkpoint
from google.colab import userdata
from huggingface_hub import login, hf_hub_download
login(userdata.get('HF_TOKEN'))

import os
os.makedirs('checkpoints/sam3', exist_ok=True)
ckpt = hf_hub_download('facebook/sam3', 'sam3.pt', local_dir='./checkpoints/sam3', token=userdata.get('HF_TOKEN'))
os.symlink('./checkpoints/sam3/sam3.pt', './checkpoints/sam3/sam3_hiera_large.pt')
os.makedirs('./checkpoints/sam3/checkpoints', exist_ok=True)
os.symlink('./checkpoints/sam3/sam3.pt', './checkpoints/sam3/checkpoints/sam3_hiera_tiny.pt')
print('✅ Setup complete')

In [ ]:
# 2. Backend (Leave running)
import nest_asyncio, subprocess, time, requests
nest_asyncio.apply()

%cd /content/vtrack-ai-studio/backend
!pkill -f uvicorn || true
time.sleep(2)

proc = subprocess.Popen(['uvicorn', 'server:app', '--host', '0.0.0.0', '--port', '8000'])
print(f'🚀 Backend PID: {proc.pid}')
time.sleep(5)

r = requests.get('http://localhost:8000/api/health', timeout=10)
print('✅ Backend:', r.json())

# KEEP THIS CELL RUNNING
proc.wait()

In [ ]:
# 3. Frontend (Run LAST - Get UI link)
%cd /content/vtrack-ai-studio
!echo 'VITE_API_URL=http://localhost:8000' > .env
!npm i --silent

print('🚀 Vite starting...')
print('📱 Colab will show UI link above 👆')
!npm run dev -- --host 0.0.0.0 --port 4173